In [159]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import pearsonr

In [288]:
train = pd.read_csv('observations_train.csv')
test = pd.read_csv('observations_test.csv')
series = pd.read_csv('series.csv',encoding= 'unicode_escape')
a = series[series['frequency'] == 'Daily']
list(a['series_id'])

['AAA10Y',
 'BAA10Y',
 'DEXCHUS',
 'DEXJPUS',
 'DEXUSEU',
 'DEXUSUK',
 'DFII10',
 'DFII20',
 'DFII30',
 'DFII5',
 'DFII7',
 'DLTIIT',
 'DPCREDIT',
 'DPRIME',
 'EFFR',
 'EFFRVOL',
 'OBFR',
 'OBFRVOL',
 'SOFR',
 'SOFRVOL',
 'T10YIE',
 'TEDRATE']

In [75]:
time = train['date'].apply(lambda x: x.split(' ')[0])
time = time.apply(lambda x: datetime.strptime(x, "%m-%d-%Y"))
train['date'] = time
train

,series_id,date,value
84020,SP500,2011-02-14,1332.32
84021,SP500,2011-02-15,1328.01
84022,SP500,2011-02-16,1336.32
84023,SP500,2011-02-17,1340.43
84024,SP500,2011-02-18,1343.01
...,...,...,...
85810,SP500,2017-12-25,NaN
85811,SP500,2017-12-26,2680.50
85812,SP500,2017-12-27,2682.62
85813,SP500,2017-12-28,2687.54


In [165]:
def person_calculation(sumx, sumy, sumxy, sumx2, sumy2, length):
    a = length * sumxy - (sumx * sumy)
    b = np.sqrt(((length * sumx2) - (sumx ** 2)) - ((length * sumy2) - (sumy**2)))
    return a/b
    
df = train 
train = train[train['value'].notna()]
    
ids = series[(series['frequency'] == 'Daily')]['series_id']
x = set(train[train['series_id'] == 'SP500']['date'])
person = []
for id in train['series_id'].unique(): 
    y = set(train[train['series_id'] == id]['date'])
    z = x.intersection(y)
    
    if(len(z) < 2): 
        continue
        
    boo = (train[train['series_id'] == 'SP500']['date'].isin(z))
    index = boo[boo == True].index
    spdf = train.loc[index, :]['value']

    boo = train[train['series_id'] == id]['date'].isin(z)
    index = boo[boo == True].index
    iddf = train.loc[index, :]['value']
    
    corr, _ = pearsonr(spdf, iddf)
    person.append([corr,id, len(z)])
person

[[-0.7023413601213642, 'AAA10Y', 1718],
 [-0.8450462978428144, 'ASEANTOT', 53],
 [-0.6401682040084496, 'BAA10Y', 1718],
 [-0.9684085185638347, 'CUUR0000SA0R', 53],
 [0.45168741365816956, 'DEXCHUS', 1716],
 [-0.7314356967121483, 'DEXUSEU', 1716],
 [-0.6878921959418713, 'DEXUSUK', 1716],
 [0.4073609779467775, 'DFII10', 1719],
 [0.07745235952728308, 'DFII20', 1719],
 [-0.05934880863327176, 'DFII30', 1719],
 [0.7337786964605856, 'DFII5', 1719],
 [0.604003921906159, 'DFII7', 1719],
 [0.14900007614999977, 'DLTIIT', 1719],
 [0.7168543907470264, 'DPCREDIT', 1732],
 [0.7172769770840273, 'DPRIME', 1719],
 [-0.44954251901233533, 'EECTOT', 53],
 [0.7145193865168538, 'EFFR', 1719],
 [0.8186719483607902, 'EFFRVOL', 461],
 [0.1057419142549715, 'FRGSHPUSM649NCIS', 53],
 [0.8538997469373831, 'GVIPT50002S', 53],
 [0.9095365759033556, 'GVIPT50030S', 53],
 [0.9125968297609531, 'GVIPT51000S', 53],
 [0.9284121075721483, 'GVIPT51100S', 53],
 [0.9154084901361863, 'GVIPT51110S', 53],
 [0.9656876457464233, 'GVI

In [222]:
data1 = pd.read_csv('1_raw_training_horizontal.csv')
data2 = pd.read_csv('2_raw_training_horizontal_without_weekends.csv')
data3 = pd.read_csv('3_raw_training_horizontal_without_weekend_naively_filled.csv')
data4 = pd.read_csv('4_change_date_range_cut_missing_gt_5pct(1).csv')
data = pd.read_csv('5_fill_missing_value.csv')
data5 = data
data5 = data5.iloc[:,2:].diff()
data5 = data5.iloc[1:, :]
series = np.sum(data5, axis = 0)
data5 = data5.divide(series, axis=1)

In [ ]:
normalized diff

In [221]:
pearson_fill = {key: [] for key in data5.columns[2:]}
for i in range(7):
    lst = []
    for col in data5.columns[2:]: 
        if col == 'SP500': 
            continue
        corr, _ = pearsonr(data5['SP500'][i:], data5[col][:len(data5['SP500'])-i])
        pearson_fill[col].append(corr)

pearson_fill

{'BAA10Y': [0.023460527560245917,
  -0.059857920831205086,
  -0.03899863045793251,
  0.026158680404571512,
  0.02418599356699592,
  -0.00884217678172876,
  0.030081941144937018],
 'BUSAPPWNSAUS': [0.005220354672927705,
  0.0048412947224335,
  0.011920686390581754,
  -0.021948504792012564,
  0.04626236059524574,
  -0.019905454414673125,
  0.02883374543888096],
 'BUSAPPWNSAUSYY': [-0.012668043520523477,
  -0.009006672473871557,
  -0.01583031722690576,
  0.01334199125704547,
  0.007697926351790888,
  0.01192137370870263,
  0.030533203402568588],
 'CBUSAPPWNSAUS': [-0.004980014627999334,
  -0.009550224772480045,
  -0.0036948150901423313,
  0.0164136861894619,
  -0.04614806719076563,
  0.02471182082879059,
  -0.02583331344113909],
 'CBUSAPPWNSAUSYY': [-0.009764411583900042,
  -0.008453787858749465,
  -0.014316731031479915,
  0.014062014153195263,
  0.008861272672663427,
  0.010239697168463601,
  0.033174067267554036],
 'CUUR0000SA0R': [0.010497512309581547,
  0.012816807230544956,
  -0.0302

In [269]:
data6 = data.iloc[:,2:].diff()
#data6 = data6.iloc[1:, :]
data6 = data.iloc[:,2:].pct_change()
data6 = data6.iloc[1:, :]
data5 = data6
a = data6.isna().sum()

data5.columns[2:]
lsts = ['BAA10Y', 'BUSAPPWNSAUS', 'BUSAPPWNSAUSYY', 'CBUSAPPWNSAUS','CBUSAPPWNSAUSYY', 'CUUR0000SA0R', 'DEXCHUS', 'DEXUSEU', 'DEXUSUK', 'DFII30', 'DPCREDIT','DPRIME', 'EECTOT', 'EFFR', 'FRGSHPUSM649NCIS', 'GVIPT50002S',
       'GVIPT50030S', 'GVIPT51000S', 'GVIPT51100S', 'GVIPT51110S',\
       'GVIPT51120S', 'GVIPT51200S', 'GVIPT52000S', 'GVIPT52100S',\
       'GVIPT521A3S', 'GVIPT52300S', 'GVIPT54000S', 'GVIPT54100S',\
       'GVIPT54200S', 'GVIPT54220S', 'IOER', 'IORR',\
       'MEDCPIM158SFRBCLE', 'MORTGAGE15US', 'MORTGAGE30US', 'PCUADLVWRADLVWR',\
       'PCUAINFOAINFO', 'PCUAMUMAMUM', 'PCUARETTRARETTR', 'PCUASHCASHC',\
       'PCUASTDSVASTDSV', 'PCUATRADEATRADE', 'PCUATRANSATRANS',\
       'PCUATRNWRATRNWR', 'PCUATTDSVATTDSV', 'PCUAWHLTRAWHLTR', 'PCUOMFGOMFG',\
       'PCUOMINOMIN', 'SFTPAGRM158SFRBSF', 'SP500', 'T10YIE', 'TEDRATE',\
       'TLAACBW027NBOG', 'TLBACBW027NBOG', 'TSIFRGHT', 'UNRATE', 'WLEMUINDXD']

In [278]:
#percentage of change
len(data5['SP500'])

1794

In [280]:
pearson_fill = {key: [] for key in lsts}
for i in range(7):
    lst = []
    for col in lsts: 
        if col == 'SP500': 
            continue
        corr, _ = pearsonr(data5['SP500'][i:], data5[col][:len(data5['SP500'])-i])
        pearson_fill[col].append(corr)
pearson_fill

series

AAA10Y              -0.540
ASEANTOT            -5.200
BAA10Y              -0.760
BUSAPPWNSAUS       160.000
BUSAPPWNSAUSYY       8.120
                    ...   
TLAACBW027NBOG    4874.997
TLBACBW027NBOG    4368.730
TSIFRGHT            24.100
UNRATE              -4.900
WLEMUINDXD          -7.370
Length: 65, dtype: float64